In [105]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

data = ImageFolder(root = "./image", transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((50,50))]))
data_test = ImageFolder(root = "./image_test", transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((50,50))]))

In [106]:
data.classes, data.class_to_idx, len(data.imgs)
test=data.imgs

In [107]:
import numpy as np
import pandas as pd
pd.Series(np.array(test)[:,1].astype("uint8")).value_counts()
# 대략 1.5:1

0    118
1     78
Name: count, dtype: int64

In [108]:
from torch.utils.data import DataLoader, WeightedRandomSampler 
import numpy as np 
import torch

num_data = 118
min_weight = 0.001
max_weight = 10000  

log_weights = np.linspace(np.log(min_weight), np.log(max_weight), num_data)
weights = np.exp(log_weights)

num_data = 78
min_weight = 0.001
max_weight = 10000  

log_weights = np.linspace(np.log(min_weight), np.log(max_weight), num_data)
weights2 = np.exp(log_weights)

#================================================================================================
# 
weights = [1/118 for _ in range(118)]
weights2 = [1/78 for _ in range(78)]


weights=np.hstack((weights,weights2))

In [109]:
sampler = WeightedRandomSampler(torch.DoubleTensor(list(weights)), 118+78, replacement=False)
data_loader = DataLoader(data, batch_size=8, sampler=sampler)
data_loader2 = DataLoader(data_test, batch_size=10)

In [110]:
for i,j in data_loader:
    print(i.shape, j) 

c:\Users\mathn\.conda\envs\Torch_PY38\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([8, 3, 50, 50]) tensor([0, 0, 1, 1, 1, 0, 1, 1])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 1, 0, 1, 1, 1, 1])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 0, 0, 0, 0, 1])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 0, 1, 1, 0, 1, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 0, 1, 0, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 1, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 1, 1, 0, 1, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 0, 0, 1, 0, 0, 1])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 0, 0, 0, 0, 1])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 1, 1, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 0, 1, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 0, 1, 1, 1, 1])
torch.Size([8, 3, 50, 50]) tensor([1, 1, 1, 0, 1, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 1, 0, 0, 1, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 0, 0, 1, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 0, 1, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 0,

In [111]:
# CNN 모델 클래스 생성
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [112]:
# 모델 객체 생성, Adam 옵티마이저 생성
import torch.optim as optim

model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [113]:
import torchmetrics.functional as metrics

for epoch in range(10):
    for i, data in enumerate(data_loader):
        x, y = data
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            pass
            #print(epoch, i, loss.item())
    
    with torch.no_grad() : 
        for i, data in enumerate(data_loader2):
            x, y = data
            output = model(x)
            print("정확도 : ",metrics.accuracy(output.argmax(dim=1), y, task="binary"))
        

정확도 :  tensor(1.)
정확도 :  tensor(0.7000)
정확도 :  tensor(0.)
정확도 :  tensor(1.)
정확도 :  tensor(0.7000)
정확도 :  tensor(0.)
정확도 :  tensor(1.)
정확도 :  tensor(0.9000)
정확도 :  tensor(0.4444)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(0.5556)
정확도 :  tensor(1.)
정확도 :  tensor(0.9000)
정확도 :  tensor(0.4444)
정확도 :  tensor(0.8000)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(0.7778)
정확도 :  tensor(1.)
정확도 :  tensor(0.8000)
정확도 :  tensor(0.2222)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(0.7778)
정확도 :  tensor(1.)
정확도 :  tensor(1.)
정확도 :  tensor(0.8889)


In [114]:
import cv2
import torch
def anya_bekki_classification(filepath):
    img=cv2.imread(filepath)

    # 해당 이미지를 50,50으로 만들기
    img = cv2.resize(img, (50,50))
    cv2.imshow("img", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

    if not model(torch.FloatTensor(img.reshape(1,3,50,50))).argmax() :
        print("아냐~")
    else : 
        print("베키.")

In [115]:
filepath="./image_test/bekki/B.jpg"
anya_bekki_classification(filepath)

베키.


In [117]:
filepath="./image_test/anya/test1.png"
anya_bekki_classification(filepath)

아냐~
